In [69]:
import sys
if 'google.colab' in sys.modules:
    import os
    from google.colab import files
    # just check if we already uploaded, may we restart the runtime and run all cells
    if not os.path.isfile('mobian_data.xlsx'):
        uploaded = files.upload()

import pandas as pd
import numpy as np
import pyomo.environ as pyo

In [70]:
data = pd.read_excel('mobian_data.xlsx', sheet_name=None, index_col=0)

In [85]:
demand = data['v'].reset_index()

distances_jh = data['d_jh'].reset_index()
distances_hp = data['d_hp'].reset_index()
distances_jp = data['d_jp'].reset_index()

time_jh = data['c_jh'].reset_index()
time_hp = data['b_hp'].reset_index()
time_jp = data['c_jp'].reset_index()

junctions = data['junctions']
hubs      = data['hubs']
pois      = data['pois']

junctions_index = set(junctions.index)
hubs_index      = set(hubs.index)
pois_index      = set(pois.index)

parameters = data['single_parameters'].reset_index().to_dict('records')[0]

demand.columns = list(pois.index)
demand         = demand.T.groupby(axis=0, level=0).mean().T
demand.index   = list(junctions.index)


distances_jh.columns = list(hubs.index)
distances_jh         = distances_jh.T.groupby(axis=0, level=0).mean().T
distances_jh.index   = list(junctions.index)

distances_hp.columns = list(pois.index)
distances_hp         = distances_hp.T.groupby(axis=0, level=0).mean().T
distances_hp.index   = list(hubs.index)
distances_hp         = distances_hp.groupby(axis=0, level=0).mean()

distances_jp.columns = list(pois.index)
distances_jp         = distances_jp.T.groupby(axis=0, level=0).mean().T
distances_jp.index   = list(junctions.index)


time_jh.columns = list(hubs.index)
time_jh         = time_jh.T.groupby(axis=0, level=0).mean().T
time_jh.index   = list(junctions.index)

time_hp.columns = list(pois.index)
time_hp         = time_hp.T.groupby(axis=0, level=0).mean().T
time_hp.index   = list(hubs.index)
time_hp         = time_hp.groupby(axis=0, level=0).mean()

time_jp.columns = list(pois.index)
time_jp         = time_jp.T.groupby(axis=0, level=0).mean().T
time_jp.index   = list(junctions.index)

pois = pois.groupby(axis=0, level=0).mean()
hubs = hubs.groupby(axis=0, level=0).mean()


In [86]:
hubs

,Longitude,Latitude
Name,,
Berchvliet,4.800172,52.390900
Byzantium,4.879838,52.361973
Car Parking AH - XL,4.801915,52.359415
Carnapstraat,4.839240,52.354200
Centrum Oosterdok,4.909145,52.376248
...,...,...
Willem van Weldammelaan,4.875717,52.330108
Willemspoort,4.884422,52.384951
Winkelcentrum Mosveld,4.912210,52.391255


In [88]:
def Mobian(demand, junctions, hubs, pois, parameters, 
           distances_jp, distances_hp, distances_jh, 
           time_jp, time_hp, time_jh, solver='glpk'):
    
    model = pyo.ConcreteModel('Mobian')

    #Set index initialize
    model.J = pyo.Set(initialize = junctions.index) #junctions
    model.H = pyo.Set(initialize = hubs.index) #hubs
    model.P = pyo.Set(initialize = pois.index) #pois

    #Parameters
    # single parameters
    model.Delta = pyo.Param(mutable = False, default = parameters['Delta'])
    model.tau   = pyo.Param(mutable = False, default = parameters['tau'])
    model.T     = pyo.Param(mutable = False, default = parameters['T'])
    model.D     = pyo.Param(mutable = False, default = parameters['D'])
    model.U     = pyo.Param(mutable = False, default = parameters['U'])

    #distance from a location to another
    model.d_jp = pyo.Param(model.J, model.P, 
                        initialize = lambda model, j, p: distances_jp.loc[j, p])

    model.d_jh = pyo.Param(model.J, model.H, 
                        initialize = lambda model, j, h: distances_jh.loc[j, h])

    model.d_hp = pyo.Param(model.H, model.P, 
                        initialize = lambda model, h, p: distances_hp.loc[h, p])

    #car/bike time from a location to another
    model.t_jp = pyo.Param(model.J, model.P, 
                        initialize = lambda model, j, p: time_jp.loc[j, p])

    model.t_jh = pyo.Param(model.J, model.H, 
                        initialize = lambda model, j, h: time_jh.loc[j, h])

    model.t_hp = pyo.Param(model.H, model.P,
                        initialize = lambda model, h, p: time_hp.loc[h, p])

    #demand from a junction to a poi
    model.n_jp = pyo.Param(model.J, model.P,
                        initialize = lambda model, j, p: demand.loc[j, p])
    #variables
    model.x = pyo.Var(model.H, within = pyo.Binary)
    model.z = pyo.Var(model.J, model.P, within = pyo.Binary)

    # Objective function
    @model.Objective(sense=pyo.maximize)
    def obj(model):
        return pyo.quicksum(pyo.quicksum(model.n_jp[j, p]*model.z[j, p] for j in model.J) for p in model.P)

    @model.Constraint(model.J, model.P)
    def service_used_condition(model, j, p):
        temp = pyo.quicksum(model.x[h] for h in model.H 
                            if 
                            (model.t_jh[j,h]+model.t_hp[h,p]-model.t_jp[j,p] <= model.Delta) and
                            (model.t_hp[h, p]                                <= model.T)     and
                            (model.d_hp[h, p]                                >= model.D)     and
                            (model.d_jp[j, p]-model.d_jh[j, h]               >= model.tau)
                            )
        return model.z[j, p] <= temp 
    
    @model.Constraint()
    def max_hub_built(model):
        return pyo.quicksum(model.x[h] for h in model.H) <= model.U

    def solve():
        result = pyo.SolverFactory(solver).solve(model)
        return result
    
    result = solve()

    return model, result


In [89]:
model, result = Mobian(demand, junctions, hubs, pois, parameters,
               distances_jp, distances_hp, distances_jh, 
               time_jp, time_hp, time_jh
               )


In [90]:
for i in model.x:
    if model.x[i].value > 0:
        print(str(model.x[i]), model.x[i].value)


x[Louwesweg] 1.0
x[Mobihub Oost] 1.0
x[Mobihub Sloterdijk] 1.0
x[Parking117] 1.0
x[Parking134] 1.0
x[Parking55] 1.0
x[Parking90] 1.0


In [91]:
print(pyo.quicksum(pyo.quicksum(model.n_jp[j, p]*model.z[j, p].value for j in model.J) for p in model.P))

9482.337809899998
